In [1]:
import pandas as pd
import config_prod as config
#import json
from datetime import date, timedelta
import phood_api

In [2]:
names = pd.read_excel('Phood Solutions List Names.xlsx')
names=names.drop([0,1,2])
names=names.iloc[:,0:2]

names.columns = ['PLU', 'Item_Names']
names.reset_index(drop=True)
ndic = names.set_index('PLU').to_dict('index')

In [13]:
locations = ['dedham', 'hingham', 'westford', 'westhartford', 'cranston']
unnamedplu = []
masterlist=[]
#print(ndic)
locationinfo = {51: {'name': 'Dedham'}, 52: {'name': 'Hingham'}, 53: {'name': 'Westford'}, 54: {'name': 'West hartford'}, 55: {'name': 'Cranston'}}
for location in locations:
    phoodServer = phood_api.PhoodAPI(config.base_url)
    username = location
    res = phoodServer.login(username, config.password).json()
    endDate = date.today() #- timedelta(days=date.today().weekday())
    startDate = date(2019,9, 29)
    #endDate = date.today()- timedelta(days=1)
    #startDate = endDate - timedelta(days=6)
    res = phoodServer.get_food_logs(startDate, endDate).json()
    PLUEntries = filter(lambda x: x['clientId'], res)
    orderedEntries = list(sorted(PLUEntries, key=lambda x: x["loggedTime"]))
    hotBar,saladBar,soupBar = set(),set(),set()
    res = phoodServer.getPersistentItems().json()
    for pItem in res:
        if pItem['station'].lower().startswith('hot bar'):
            hotBar.add(pItem['clientId'])
        elif pItem['station'].lower().startswith('salad bar'):
            saladBar.add(pItem['clientId'])
        elif pItem['station'].lower().startswith('soup'):
            soupBar.add(pItem['clientId'])
        else:
            print(f'unclassified item {pItem}')
    for item in orderedEntries:
        item['clientId'] = item['clientId'].replace("-", "")
        #print(item['clientId'])
        if item['itemName'] in ndic.keys():
            item['itemName'] = ndic[int(item['clientId'])]['Item_Names']
        else:
            unnamedplu.append(item)
        #print(item['Item_Names'])
        if item['clientId'] in hotBar:
            item['station'] = 'HB'
        elif item['clientId'] in saladBar:
            item['station'] = 'SB'
        elif item['clientId'] in soupBar:
            item['station'] = 'SO'
        item['Location'] = locationinfo[item['locationId']]['name']
        
    res = phoodServer.getPanInfo().json()
    panInfo = {}
    for pan in res:
        panInfo[pan["id"]] = {"name": pan["name"], "weight": pan["weightQuantity"]}
    #print(panInfo)
    for item in orderedEntries:
        item['Served']=item['Saved']=item['Sold']=item['Shrink']=0
        if 'panId' in item and item['panId']:
            panWeight = panInfo[item['panId']]['weight']
            item['Pan_Name'] = panInfo[item['panId']]['name']
            
        else:
            panWeight = item['panWeight'] 
        if item['actionTaken']=='Served':
            item['Served'] = +item['quantity'] - panWeight
        if item['actionTaken']=='Discarded':
            item['Shrink'] = +item['quantity'] - panWeight
        if item['actionTaken']=='Saved':
            item['Saved'] = +item['quantity'] - panWeight
        item['Sold'] = item['Served'] - item['Saved'] - item['Shrink']
    masterlist.extend(orderedEntries)

In [14]:
#print(startDate)
#print(endDate)

In [15]:
#masterlist

In [16]:
#type(masterlist)

In [17]:
raw = pd.DataFrame(masterlist) 

In [18]:
#raw[raw.clientId == '95501']

In [19]:
raw

,dateProduced,id,itemName,itemType,clientId,loggedTime,meal,actionTaken,actionReason,quantity,...,unit,station,employeeName,locationId,Location,Served,Saved,Sold,Shrink,Pan_Name
0,2019-09-28,41385,Romaine Lettuce,Menu Item,46000055863,2019-09-29T06:10:55.000Z,Breakfast,Served,Start,10.38380,...,Lbs,SB,None,51,Dedham,7.14280,0.0,7.14280,0.00000,NaN
1,2019-09-28,41386,Romaine Lettuce,Menu Item,46000055863,2019-09-29T06:11:11.000Z,Breakfast,Served,Start,9.74443,...,Lbs,SB,None,51,Dedham,6.50343,0.0,6.50343,0.00000,NaN
2,2019-09-28,41387,Baby Spinach,Menu Item,46000055923,2019-09-29T06:11:30.000Z,Breakfast,Served,Start,6.52568,...,Lbs,SB,None,51,Dedham,3.28468,0.0,3.28468,0.00000,NaN
3,2019-09-28,41388,Spring Mix,Menu Item,46000063050,2019-09-29T06:11:46.000Z,Breakfast,Served,Start,6.48159,...,Lbs,SB,None,51,Dedham,3.24059,0.0,3.24059,0.00000,NaN
4,2019-09-28,41389,Kale,Menu Item,46000055842,2019-09-29T06:20:21.000Z,Breakfast,Served,Start,7.87050,...,Lbs,SB,None,51,Dedham,4.62950,0.0,4.62950,0.00000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29814,2019-11-04,71594,Buffalo Wings,Menu Item,46000063709,2019-11-04T21:08:36.000Z,Breakfast,Discarded,Over Produced,7.14298,...,Lbs,HB,None,55,Cranston,0.00000,0.0,-5.24298,5.24298,4 inch half hotel pan
29815,2019-11-04,71596,Mashed Potatoes,Menu Item,46000057474,2019-11-04T21:09:28.000Z,Breakfast,Discarded,Over Produced,4.23288,...,Lbs,HB,None,55,Cranston,0.00000,0.0,-3.28288,3.28288,2 inch half hotel pan
29816,2019-11-04,71597,Steamed Broccoli,Menu Item,46000064891,2019-11-04T21:10:02.000Z,Breakfast,Discarded,Over Produced,2.77782,...,Lbs,HB,None,55,Cranston,0.00000,0.0,-0.87782,0.87782,4 inch half hotel pan
29817,2019-11-04,71599,White Quinoa,Menu Item,46000055911,2019-11-04T21:10:52.000Z,Breakfast,Discarded,Over Produced,6.23908,...,Lbs,HB,None,55,Cranston,0.00000,0.0,-4.26908,4.26908,Soup Bain Marie


In [20]:
rawfile=raw

In [21]:
#raw.isnull().sum()

In [22]:
raw.columns = ['Date_Produced', 'Log_ID', 'Item_Name', 'Item_Type', 'PLU', 'Logged_Time', 'Meal', 'Action_Taken', 'Action_Reason', 'Quantity', 'Pan_ID', 'Pan_Weight', 'Unit', 'Station', 'Employee_Name', 'Location_ID','Location', 'Served','Saved','Sold','Shrink','Pan_Name']

In [540]:
#raw

In [541]:
import pandas_profiling
#pandas_profiling.ProfileReport(raw)

## Remove Unit, Meal, Item Type, Employee Name columns

In [542]:
raw = raw.drop(columns=['Unit','Meal','Item_Type','Employee_Name'])

In [543]:
raw['Date_Produced'] = pd.to_datetime(raw['Date_Produced'])
raw['Logged_Time'] = pd.to_datetime(raw['Logged_Time'])

In [544]:
raw['Logged_Date'] = raw['Logged_Time'].dt.date
#raw['Logged_Time'] = raw['Logged_Time'].dt.time

In [545]:
#raw

In [546]:
#raw.isnull().sum()

In [547]:
#raw[raw.isna().any(axis=1)].PLU.unique()

In [548]:
#raw.PLU.unique()

In [549]:
raw['Actual_Weight'] = raw['Quantity'] - raw['Pan_Weight']

In [550]:
#raw

In [551]:
#raw.PanWeight.unique()
#pd.Series(raw['Pan_Weight']).unique()

In [552]:
raw = raw.drop(columns=['Log_ID','Date_Produced','Logged_Time','Action_Taken','Action_Reason','Pan_ID','Pan_Weight','Pan_Name'])

In [553]:
#raw

In [554]:
#pandas_profiling.ProfileReport(raw)

In [555]:
file = raw.groupby(['Location_ID','Station','Logged_Date','PLU','Item_Name','Location'],as_index=False).sum()
#file

In [556]:
file = file.reindex(columns = ['Logged_Date', 'Station', 'PLU', 'Item_Name', 'Served','Saved','Shrink','Sold','Quantity','Actual_Weight','Location_ID','Location'])


In [557]:
negativeweights = file[(file.Served < 0) | (file.Saved < 0) | (file.Shrink < 0)]

In [558]:
#negativeweights

In [559]:
file = file[(file['Served'] >= 0) & (file['Saved'] >= 0) & (file['Shrink'] >= 0) & (file['Sold'] >=0)].reset_index(drop=True) 
file = file.round(2)
file['PLU'] = pd.to_numeric(file.PLU)
#file


In [560]:
file0 = file.drop(columns=['Quantity','Actual_Weight'])
file0

,Logged_Date,Station,PLU,Item_Name,Served,Saved,Shrink,Sold,Location_ID,Location
0,2019-09-29,HB,22593200000,Maria's Beef Meatballs,25.13,0.00,1.45,23.68,51,Dedham
1,2019-09-29,HB,26747000000,Cajun Sweet Potato Wedges,12.26,0.00,0.68,11.58,51,Dedham
2,2019-09-29,HB,46000001661,Fried Plantains,4.72,0.00,0.00,4.72,51,Dedham
3,2019-09-29,HB,46000005678,Chicken Cacciatore,28.78,5.25,1.28,22.25,51,Dedham
4,2019-09-29,HB,46000007582,Breaded Chicken Tenders,58.23,0.00,5.69,52.54,51,Dedham
...,...,...,...,...,...,...,...,...,...,...
8649,2019-10-28,SB,46000059670,Sesame Noodles,4.91,0.00,0.00,4.91,55,Cranston
8650,2019-10-29,SB,46000010550,Seasonal Pick Mushrooms,1.47,0.00,0.00,1.47,55,Cranston
8651,2019-10-29,SB,46000055863,Romaine Lettuce,3.08,0.00,3.07,0.01,55,Cranston
8652,2019-10-29,SB,46000055935,Organic Spring Mix,2.74,0.00,0.70,2.04,55,Cranston


In [561]:
file0.columns = ['Date', 'Venue','PLU', 'Item Name', 'Served', 'Saved', 'Shrink', 'Sold', 'Location_ID', 'Store']
file0 = file0.reindex(columns = ['Date','Store','Venue','PLU', 'Item Name', 'Served', 'Saved', 'Shrink', 'Sold', 'Location_ID'])
file0

,Date,Store,Venue,PLU,Item Name,Served,Saved,Shrink,Sold,Location_ID
0,2019-09-29,Dedham,HB,22593200000,Maria's Beef Meatballs,25.13,0.00,1.45,23.68,51
1,2019-09-29,Dedham,HB,26747000000,Cajun Sweet Potato Wedges,12.26,0.00,0.68,11.58,51
2,2019-09-29,Dedham,HB,46000001661,Fried Plantains,4.72,0.00,0.00,4.72,51
3,2019-09-29,Dedham,HB,46000005678,Chicken Cacciatore,28.78,5.25,1.28,22.25,51
4,2019-09-29,Dedham,HB,46000007582,Breaded Chicken Tenders,58.23,0.00,5.69,52.54,51
...,...,...,...,...,...,...,...,...,...,...
8649,2019-10-28,Cranston,SB,46000059670,Sesame Noodles,4.91,0.00,0.00,4.91,55
8650,2019-10-29,Cranston,SB,46000010550,Seasonal Pick Mushrooms,1.47,0.00,0.00,1.47,55
8651,2019-10-29,Cranston,SB,46000055863,Romaine Lettuce,3.08,0.00,3.07,0.01,55
8652,2019-10-29,Cranston,SB,46000055935,Organic Spring Mix,2.74,0.00,0.70,2.04,55


In [562]:
file1 = file0[file0['Location_ID']==51]
file2 = file0[file0['Location_ID']==52]
file3 = file0[file0['Location_ID']==53]
file4 = file0[file0['Location_ID']==54]
file5 = file0[file0['Location_ID']==55]
file0 = file0.drop(columns=['Location_ID'])
file1 = file1.drop(columns=['Location_ID'])
file2 = file2.drop(columns=['Location_ID'])
file3 = file3.drop(columns=['Location_ID'])
file4 = file4.drop(columns=['Location_ID'])
file5 = file5.drop(columns=['Location_ID'])

#file1

In [563]:
rawfile

,Date_Produced,Log_ID,Item_Name,Item_Type,PLU,Logged_Time,Meal,Action_Taken,Action_Reason,Quantity,...,Unit,Station,Employee_Name,Location_ID,Location,Served,Saved,Sold,Shrink,Pan_Name
0,2019-09-28,41385,Romaine Lettuce,Menu Item,46000055863,2019-09-29T06:10:55.000Z,Breakfast,Served,Start,10.38380,...,Lbs,SB,None,51,Dedham,7.14280,0.0,7.14280,0.00000,NaN
1,2019-09-28,41386,Romaine Lettuce,Menu Item,46000055863,2019-09-29T06:11:11.000Z,Breakfast,Served,Start,9.74443,...,Lbs,SB,None,51,Dedham,6.50343,0.0,6.50343,0.00000,NaN
2,2019-09-28,41387,Organic Roasted Sweet Potatoes,Menu Item,46000055923,2019-09-29T06:11:30.000Z,Breakfast,Served,Start,6.52568,...,Lbs,SB,None,51,Dedham,3.28468,0.0,3.28468,0.00000,NaN
3,2019-09-28,41388,Spring Mix,Menu Item,46000063050,2019-09-29T06:11:46.000Z,Breakfast,Served,Start,6.48159,...,Lbs,SB,None,51,Dedham,3.24059,0.0,3.24059,0.00000,NaN
4,2019-09-28,41389,Kale,Menu Item,46000055842,2019-09-29T06:20:21.000Z,Breakfast,Served,Start,7.87050,...,Lbs,SB,None,51,Dedham,4.62950,0.0,4.62950,0.00000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26817,2019-10-24,65768,Bourbon Wings,Menu Item,46000060413,2019-10-25T21:19:27.000Z,Breakfast,Discarded,Over Produced,5.90839,...,Lbs,HB,None,55,Cranston,0.00000,0.0,-4.95839,4.95839,2 inch half hotel pan
26818,2019-10-27,67040,Bourbon Wings,Menu Item,46000060413,2019-10-27T21:06:58.000Z,Breakfast,Discarded,Over Produced,0.00000,...,Lbs,HB,None,55,Cranston,0.00000,0.0,0.95000,-0.95000,2 inch half hotel pan
26819,2019-10-28,67350,Mashed Garbanso,Menu Item,46000021384,2019-10-28T11:34:17.000Z,Breakfast,Served,Start,5.73202,...,Lbs,HB,None,55,Cranston,2.93202,0.0,2.93202,0.00000,2 inch full hotel pan
26820,2019-10-28,68306,Bourbon Wings,Menu Item,46000060413,2019-10-29T16:14:54.000Z,Breakfast,Served,Start,14.92530,...,Lbs,HB,None,55,Cranston,11.12530,0.0,11.12530,0.00000,6 inch full hotel pan


In [564]:
rawfile.groupby(['Item_Name','Pan_ID'],as_index=False).count()

,Item_Name,Pan_ID,Date_Produced,Log_ID,Item_Type,PLU,Logged_Time,Meal,Action_Taken,Action_Reason,...,Unit,Station,Employee_Name,Location_ID,Location,Served,Saved,Sold,Shrink,Pan_Name
0,Agrodulce Carrots,173.0,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1
1,Agrodulce Carrots,176.0,2,2,2,2,2,2,2,2,...,2,2,0,2,2,2,2,2,2,2
2,Agrodulce Carrots,177.0,8,8,8,8,8,8,8,8,...,8,8,0,8,8,8,8,8,8,8
3,Agrodulce Carrots,179.0,4,4,4,4,4,4,4,4,...,4,4,0,4,4,4,4,4,4,4
4,Agrodulce Carrots,182.0,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3759,Zucchini Planks,163.0,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1
3760,Zucchini Roasted Tomatoes,177.0,4,4,4,4,4,4,4,4,...,4,4,0,4,4,4,4,4,4,4
3761,Zucchini Roasted Tomatoes,178.0,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1
3762,Zucchini Roasted Tomatoes,179.0,3,3,3,3,3,3,3,3,...,3,3,0,3,3,3,3,3,3,3


In [565]:
ans = rawfile[rawfile['Action_Reason']=='Start']
ans = rawfile[rawfile['Location']=='Dedham']
ans = ans.groupby(['Item_Name','Pan_Name','Pan_ID','Pan_Weight']).size().to_frame('Count').reset_index()
ans = ans.sort_values(['Item_Name','Count'],ascending=False).reset_index(drop=True)
ansd = ans.groupby(['Item_Name'])['Count'].sum().to_frame('Sum').reset_index()
ans = ans.groupby('Item_Name').head(3).reset_index(drop=True)
ansd
mergedStuff = pd.merge(ans, ansd, on=['Item_Name'], how='inner')
mergedStufThe f['Freq_Percent'] = mergedStuff['Count'] / mergedStuff['Sum'] * 100
df = mergedStuff[mergedStuff['Freq_Percent']>60]
x = df['Item_Name']
df1 = mergedStuff[~mergedStuff['Item_Name'].isin(x)]
ans = df.append(df1)
ans.reset_index(drop = True)
ans = ans[ans['Freq_Percent']>15].reset_index(drop = True)
ans1 = ans.drop(['Count', 'Sum'], axis=1)
ans = rawfile[rawfile['Action_Reason']=='Start']
ans = rawfile[rawfile['Location']=='Dedham']
ans = ans.groupby(['Item_Name','Pan_Name','Pan_ID','Pan_Weight']).size().to_frame('Count').reset_index()
ans = ans.sort_values(['Item_Name','Count'],ascending=False).reset_index(drop=True)
ansd = ans.groupby(['Item_Name'])['Count'].sum().to_frame('Sum').reset_index()
ans = ans.groupby('Item_Name').head(3).reset_index(drop=True)
ansd
mergedStuff = pd.merge(ans, ansd, on=['Item_Name'], how='inner')
mergedStuff['Freq_Percent'] = mergedStuff['Count'] / mergedStuff['Sum'] * 100
df = mergedStuff[mergedStuff['Freq_Percent']>60]
x = df['Item_Name']
df1 = mergedStuff[~mergedStuff['Item_Name'].isin(x)]
ans = df.append(df1)
ans.reset_index(drop = True)
ans = ans[ans['Freq_Percent']>21].reset_index(drop = True)
ans2 = ans.drop(['Count', 'Sum'], axis=1)
ans = rawfile[rawfile['Action_Reason']=='Start']
ans = rawfile[rawfile['Location']=='Westford']
ans = ans.groupby(['Item_Name','Pan_Name','Pan_ID','Pan_Weight']).size().to_frame('Count').reset_index()
ans = ans.sort_values(['Item_Name','Count'],ascending=False).reset_index(drop=True)
ansd = ans.groupby(['Item_Name'])['Count'].sum().to_frame('Sum').reset_index()
ans = ans.groupby('Item_Name').head(3).reset_index(drop=True)
ansd
mergedStuff = pd.merge(ans, ansd, on=['Item_Name'], how='inner')
mergedStuff['Freq_Percent'] = mergedStuff['Count'] / mergedStuff['Sum'] * 100
df = mergedStuff[mergedStuff['Freq_Percent']>60]
x = df['Item_Name']
df1 = mergedStuff[~mergedStuff['Item_Name'].isin(x)]
ans = df.append(df1)
ans.reset_index(drop = True)
ans = ans[ans['Freq_Percent']>33].reset_index(drop = True)
ans3 = ans.drop(['Count', 'Sum'], axis=1)
ans = rawfile[rawfile['Action_Reason']=='Start']
ans = rawfile[rawfile['Location']=='West hartford']
ans = ans.groupby(['Item_Name','Pan_Name','Pan_ID','Pan_Weight']).size().to_frame('Count').reset_index()
ans = ans.sort_values(['Item_Name','Count'],ascending=False).reset_index(drop=True)
ansd = ans.groupby(['Item_Name'])['Count'].sum().to_frame('Sum').reset_index()
ans = ans.groupby('Item_Name').head(3).reset_index(drop=True)
ansd
mergedStuff = pd.merge(ans, ansd, on=['Item_Name'], how='inner')
mergedStuff['Freq_Percent'] = mergedStuff['Count'] / mergedStuff['Sum'] * 100
df = mergedStuff[mergedStuff['Freq_Percent']>60]
x = df['Item_Name']
df1 = mergedStuff[~mergedStuff['Item_Name'].isin(x)]
ans = df.append(df1)
ans.reset_index(drop = True)
ans = ans[ans['Freq_Percent']>33].reset_index(drop = True)
ans4 = ans.drop(['Count', 'Sum'], axis=1)
ans = rawfile[rawfile['Action_Reason']=='Start']
ans = rawfile[rawfile['Location']=='Cranston']
ans = ans.groupby(['Item_Name','Pan_Name','Pan_ID','Pan_Weight']).size().to_frame('Count').reset_index()
ans = ans.sort_values(['Item_Name','Count'],ascending=False).reset_index(drop=True)
ansd = ans.groupby(['Item_Name'])['Count'].sum().to_frame('Sum').reset_index()
ans = ans.groupby('Item_Name').head(3).reset_index(drop=True)
ansd
mergedStuff = pd.merge(ans, ansd, on=['Item_Name'], how='inner')
mergedStuff['Freq_Percent'] = mergedStuff['Count'] / mergedStuff['Sum'] * 100
df = mergedStuff[mergedStuff['Freq_Percent']>60]
x = df['Item_Name']
df1 = mergedStuff[~mergedStuff['Item_Name'].isin(x)]
ans = df.append(df1)
ans.reset_index(drop = True)
ans = ans[ans['Freq_Percent']>14].reset_index(drop = True)
ans5 = ans.drop(['Count', 'Sum'], axis=1)

In [566]:
with pd.ExcelWriter('defaultpan.xlsx') as writer:  # doctest: +SKIP
    ans1.to_excel(writer, sheet_name='Dedham_defaultpan', index = False)
    worksheet = writer.sheets['Dedham_defaultpan']
    worksheet.set_column(0, 1, 35)
    ans2.to_excel(writer, sheet_name='Hingham_defaultpan', index = False)
    worksheet = writer.sheets['Hingham_defaultpan']
    worksheet.set_column(0, 1, 35)
    ans3.to_excel(writer, sheet_name='Westford_defaultpan', index = False)
    worksheet = writer.sheets['Westford_defaultpan']
    worksheet.set_column(0, 1, 35)
    ans4.to_excel(writer, sheet_name='WestHartford_defaultpan',index = False)
    worksheet = writer.sheets['WestHartford_defaultpan']
    worksheet.set_column(0, 1, 35)
    ans5.to_excel(writer, sheet_name='Cranston_defaultpan', index = False)
    worksheet = writer.sheets['Cranston_defaultpan']
    worksheet.set_column(0, 1, 35)


In [567]:
'''ans = rawfile[rawfile['Action_Reason']=='Start']
ans = rawfile[rawfile['Location']=='Dedham']
ans = ans.groupby(['Item_Name','Pan_Name','Pan_ID','Pan_Weight']).size().to_frame('Count').reset_index()
ans = ans.sort_values(['Item_Name','Count'],ascending=False).reset_index(drop=True)
ansd = ans.groupby(['Item_Name'])['Count'].sum().to_frame('Sum').reset_index()
ans = ans.groupby('Item_Name').head(3).reset_index(drop=True)
ansd
mergedStuff = pd.merge(ans, ansd, on=['Item_Name'], how='inner')
mergedStuff['Freq_Percent'] = mergedStuff['Count'] / mergedStuff['Sum'] * 100
df = mergedStuff[mergedStuff['Freq_Percent']>60]
x = df['Item_Name']
df1 = mergedStuff[~mergedStuff['Item_Name'].isin(x)]
ans = df.append(df1)
ans.reset_index(drop = True)
ans = ans[ans['Freq_Percent']>15].reset_index(drop = True)
ans = ans.drop(['Count', 'Sum'], axis=1)
ans'''

"ans = rawfile[rawfile['Action_Reason']=='Start']\nans = rawfile[rawfile['Location']=='Dedham']\nans = ans.groupby(['Item_Name','Pan_Name','Pan_ID','Pan_Weight']).size().to_frame('Count').reset_index()\nans = ans.sort_values(['Item_Name','Count'],ascending=False).reset_index(drop=True)\nansd = ans.groupby(['Item_Name'])['Count'].sum().to_frame('Sum').reset_index()\nans = ans.groupby('Item_Name').head(3).reset_index(drop=True)\nansd\nmergedStuff = pd.merge(ans, ansd, on=['Item_Name'], how='inner')\nmergedStuff['Freq_Percent'] = mergedStuff['Count'] / mergedStuff['Sum'] * 100\ndf = mergedStuff[mergedStuff['Freq_Percent']>60]\nx = df['Item_Name']\ndf1 = mergedStuff[~mergedStuff['Item_Name'].isin(x)]\nans = df.append(df1)\nans.reset_index(drop = True)\nans = ans[ans['Freq_Percent']>15].reset_index(drop = True)\nans = ans.drop(['Count', 'Sum'], axis=1)\nans"